In [1]:
# import delle librerie
import findspark

location = findspark.find()
findspark.init(location)

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

spark = SparkSession.builder.appName("Classificazione data set Titanic").getOrCreate()

24/05/27 09:20:48 WARN Utils: Your hostname, deeplearning resolves to a loopback address: 127.0.1.1; using 147.163.26.113 instead (on interface enp6s0)
24/05/27 09:20:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/05/27 09:20:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Carichiamo i dati su due dataframe

trainDF = spark.read.csv('/home/rpirrone/data/train.csv',header=True,inferSchema=True)

testDF = spark.read.csv('/home/rpirrone/data/test.csv',header=True,inferSchema=True)

print("Training set:")
trainDF.printSchema()

print("Test set:")
testDF.printSchema()

Training set:
root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)

Test set:
root
 |-- PassengerId: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [3]:
# Calcoliamo imbarcati e sopravvisuti ed eseguiamo le statistiche descrittive dei due data set
passenger_count = trainDF.count()
survived_count = trainDF.filter("Survived == 1").count()

print(f'Passeggeri imbarcati: {passenger_count} di cui {survived_count} sopravvissuti')

trainDF.describe()['summary','Pclass','Age','SibSp','Parch','Fare'].show()

print(f'Passeggeri nel test set: {testDF.count()}')

testDF.describe()['summary','Pclass','Age','SibSp','Parch','Fare'].show()

Passeggeri imbarcati: 891 di cui 342 sopravvissuti


+-------+------------------+------------------+------------------+-------------------+-----------------+
|summary|            Pclass|               Age|             SibSp|              Parch|             Fare|
+-------+------------------+------------------+------------------+-------------------+-----------------+
|  count|               891|               714|               891|                891|              891|
|   mean| 2.308641975308642| 29.69911764705882|0.5230078563411896|0.38159371492704824| 32.2042079685746|
| stddev|0.8360712409770491|14.526497332334035|1.1027434322934315| 0.8060572211299488|49.69342859718089|
|    min|                 1|              0.42|                 0|                  0|              0.0|
|    max|                 3|              80.0|                 8|                  6|         512.3292|
+-------+------------------+------------------+------------------+-------------------+-----------------+

Passeggeri nel test set: 418


+-------+------------------+------------------+------------------+------------------+------------------+
|summary|            Pclass|               Age|             SibSp|             Parch|              Fare|
+-------+------------------+------------------+------------------+------------------+------------------+
|  count|               418|               332|               418|               418|               417|
|   mean|2.2655502392344498|30.272590361445783|0.4473684210526316|0.3923444976076555|  35.6271884892086|
| stddev|0.8418375519640503|14.181209235624424|0.8967595611217135|0.9814288785371694|55.907576179973844|
|    min|                 1|              0.17|                 0|                 0|               0.0|
|    max|                 3|              76.0|                 8|                 9|          512.3292|
+-------+------------------+------------------+------------------+------------------+------------------+



In [4]:
# Calcoliamo i valori nulli di tutte le colonne
print("Training Set:")
for column in trainDF.columns:
    print(f'{column}: {trainDF.filter(f"{column} IS NULL").count()}')

print("\n\nTest Set:")
for column in testDF.columns:
    print(f'{column}: {testDF.filter(f"{column} IS NULL").count()}')

Training Set:
PassengerId: 0
Survived: 0
Pclass: 0
Name: 0
Sex: 0
Age: 177
SibSp: 0
Parch: 0
Ticket: 0
Fare: 0
Cabin: 687
Embarked: 2


Test Set:
PassengerId: 0
Pclass: 0
Name: 0
Sex: 0
Age: 86
SibSp: 0
Parch: 0
Ticket: 0
Fare: 1
Cabin: 327
Embarked: 0


In [5]:
# Imputiamo l'età con il valor medio per tiolo riportato nel nome del passeggero
trainDF = trainDF.withColumn("Title",regexp_extract(col("Name"),".*, (.*?)\\. .*",1))

testDF = testDF.withColumn("Title",regexp_extract(col("Name"),".*, (.*?)\\. .*",1))

trainDF.select("Title").distinct().show()
testDF.select("Title").distinct().show()

+------------+
|       Title|
+------------+
|         Don|
|        Miss|
|         Col|
|         Rev|
|        Lady|
|      Master|
|         Mme|
|        Capt|
|          Mr|
|          Dr|
|         Mrs|
|         Sir|
|    Jonkheer|
|        Mlle|
|       Major|
|          Ms|
|the Countess|
+------------+

+------+
| Title|
+------+
|  Dona|
|  Miss|
|   Col|
|   Rev|
|Master|
|    Mr|
|    Dr|
|   Mrs|
|    Ms|
+------+



In [6]:
testDF.select(testDF.Name,testDF.Age).where(testDF.Title=='Dona').collect()

[Row(Name='Oliva y Ocana, Dona. Fermina', Age=39.0)]

In [7]:
testDF.select(testDF.Name,testDF.Age).where(testDF.Title=='Master').collect()

[Row(Name='Olsen, Master. Artur Karl', Age=9.0),
 Row(Name='Rice, Master. Albert', Age=10.0),
 Row(Name='Ryerson, Master. John Borie', Age=13.0),
 Row(Name='Boulos, Master. Akar', Age=6.0),
 Row(Name='Wells, Master. Ralph Lester', Age=2.0),
 Row(Name='Asplund, Master. Filip Oscar', Age=13.0),
 Row(Name='Touma, Master. Georges Youssef', Age=7.0),
 Row(Name='van Billiard, Master. Walter John', Age=11.5),
 Row(Name='Drew, Master. Marshall Brines', Age=8.0),
 Row(Name='Spedden, Master. Robert Douglas', Age=6.0),
 Row(Name='Danbom, Master. Gilbert Sigvard Emanuel', Age=0.33),
 Row(Name='Johnston, Master. William Arthur Willie""""', Age=None),
 Row(Name='Peacock, Master. Alfred Edward', Age=0.75),
 Row(Name='Aks, Master. Philip Frank', Age=0.83),
 Row(Name='Betros, Master. Seman', Age=None),
 Row(Name='van Billiard, Master. James William', Age=None),
 Row(Name='Sage, Master. William Henry', Age=14.5),
 Row(Name='Asplund, Master. Carl Edgar', Age=5.0),
 Row(Name='Palsson, Master. Paul Folke',

In [8]:
# Collassiamo i diversi titoli che possiamo giudicare come equivalenti in un sottoinsieme "normalizzato"
trainDF = trainDF.replace(\
    ['Mlle','Mme','Ms','Major','Lady','the Countess','Jonkheer','Capt'],\
    ['Miss','Miss','Miss','Col','Mrs','Mrs','Sir','Col']
    )

trainDF.select('Title').distinct().show()

testDF = testDF.replace(['Dona', 'Ms'],['Mrs','Miss'])

testDF.select('Title').distinct().show()

+------+
| Title|
+------+
|   Don|
|  Miss|
|   Col|
|   Rev|
|Master|
|    Mr|
|    Dr|
|   Mrs|
|   Sir|
+------+

+------+
| Title|
+------+
|  Miss|
|   Col|
|   Rev|
|Master|
|    Mr|
|    Dr|
|   Mrs|
+------+



In [9]:
# Calcoliamo l'età media dei passeggeri per titolo

avg_age_list = trainDF['Title', 'Age'].groupBy('Title').avg('Age').collect()

print(avg_age_list)

[Row(Title='Don', avg(Age)=40.0), Row(Title='Miss', avg(Age)=21.86), Row(Title='Col', avg(Age)=56.6), Row(Title='Rev', avg(Age)=43.166666666666664), Row(Title='Master', avg(Age)=4.574166666666667), Row(Title='Mr', avg(Age)=32.368090452261306), Row(Title='Dr', avg(Age)=42.0), Row(Title='Mrs', avg(Age)=35.981818181818184), Row(Title='Sir', avg(Age)=43.5)]


In [10]:
from math import floor

for row in avg_age_list:
    trainDF = trainDF.withColumn('Age',\
                                when((trainDF['Title']==row[0]) & (trainDF['Age'].isNull()),\
                                floor(row[1]+0.5)).otherwise(trainDF['Age']))

print(f'Age: {trainDF.filter(trainDF.Age.isNull()).count()} valori nulli')

24/05/27 09:22:04 ERROR CodeGenerator: failed to compile: org.codehaus.janino.InternalCompilerException: Compiling "GeneratedClass" in "generated.java": Code of method "hashAgg_doAggregateWithoutKey_0$(Lorg/apache/spark/sql/catalyst/expressions/GeneratedClass$GeneratedIteratorForCodegenStage1;)V" of class "org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1" grows beyond 64 KB
org.codehaus.janino.InternalCompilerException: Compiling "GeneratedClass" in "generated.java": Code of method "hashAgg_doAggregateWithoutKey_0$(Lorg/apache/spark/sql/catalyst/expressions/GeneratedClass$GeneratedIteratorForCodegenStage1;)V" of class "org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1" grows beyond 64 KB
	at org.codehaus.janino.UnitCompiler.compile2(UnitCompiler.java:366)
	at org.codehaus.janino.UnitCompiler.access$000(UnitCompiler.java:226)
	at org.codehaus.janino.UnitCompiler$1.visitCompilationUnit(UnitCompiler.java:33

Age: 0 valori nulli


In [11]:
# Effettuiamo la stessa cosa per il test set
test_avg_age_list = testDF['Title', 'Age'].groupBy('Title').avg('Age').collect()

print(test_avg_age_list)

for row in test_avg_age_list:
    testDF = testDF.withColumn('Age',\
                                when((testDF['Title']==row[0]) & (testDF['Age'].isNull()),\
                                floor(row[1]+0.5)).otherwise(testDF['Age']))

print(f'Age: {testDF.filter(testDF.Age.isNull()).count()} valori nulli')

[Row(Title='Miss', avg(Age)=21.774843750000002), Row(Title='Col', avg(Age)=50.0), Row(Title='Rev', avg(Age)=35.5), Row(Title='Master', avg(Age)=7.406470588235294), Row(Title='Mr', avg(Age)=32.0), Row(Title='Dr', avg(Age)=53.0), Row(Title='Mrs', avg(Age)=38.904761904761905)]
24/05/27 09:22:16 ERROR CodeGenerator: failed to compile: org.codehaus.janino.InternalCompilerException: Compiling "GeneratedClass" in "generated.java": Code of method "hashAgg_doAggregateWithoutKey_0$(Lorg/apache/spark/sql/catalyst/expressions/GeneratedClass$GeneratedIteratorForCodegenStage1;)V" of class "org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1" grows beyond 64 KB
org.codehaus.janino.InternalCompilerException: Compiling "GeneratedClass" in "generated.java": Code of method "hashAgg_doAggregateWithoutKey_0$(Lorg/apache/spark/sql/catalyst/expressions/GeneratedClass$GeneratedIteratorForCodegenStage1;)V" of class "org.apache.spark.sql.catalyst.expressions.GeneratedClass

Age: 0 valori nulli


In [12]:
# Imputiamo il valor medio di fare rispetto alla classe nel test set

avg_fare = testDF['Pclass','Fare'].groupBy('Pclass').avg('Fare').collect()

for row in avg_fare:
    testDF = testDF.withColumn('Fare',\
                                when((testDF['Pclass']==row[0]) & (testDF['Fare'].isNull()),row[1]).otherwise(testDF['Fare']))

print(f'Fare: {testDF.filter(testDF.Fare.isNull()).count()} valori nulli')

Fare: 0 valori nulli


In [13]:
# Imputiamo a maggioranza i valore di 'Embarked'
trainDF.groupBy('Embarked').count().show()

testDF.groupBy('Embarked').count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|    null|    2|
|       C|  168|
|       S|  644|
+--------+-----+

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   46|
|       C|  102|
|       S|  270|
+--------+-----+



In [14]:
# Imputiamo solo nel training set
trainDF = trainDF.na.fill({'Embarked': 'S'})

In [15]:
# Creiamo la colonna FamilySize come SibSp + Parch +1

trainDF = trainDF.withColumn('FamilySize',col('SibSp')+col('Parch')+1)
testDF = testDF.withColumn('FamilySize',col('SibSp')+col('Parch')+1)

trainDF.groupBy('FamilySize').count().show()
testDF.groupBy('FamilySize').count().show()

+----------+-----+
|FamilySize|count|
+----------+-----+
|         1|  537|
|         6|   22|
|         3|  102|
|         5|   15|
|         4|   29|
|         8|    6|
|         7|   12|
|        11|    7|
|         2|  161|
+----------+-----+

+----------+-----+
|FamilySize|count|
+----------+-----+
|         1|  253|
|         6|    3|
|         3|   57|
|         5|    7|
|         4|   14|
|         8|    2|
|         7|    4|
|        11|    4|
|         2|   74|
+----------+-----+



In [16]:
# creiamo una apposita feature Alone per supportare una classificazione corretta confortati dal reale svolgimento dei fatti

trainDF = trainDF.withColumn('Alone',lit(0))
trainDF = trainDF.withColumn('Alone',\
                            when(trainDF.FamilySize == 1,1).otherwise(trainDF.Alone))

testDF = testDF.withColumn('Alone',lit(0))
testDF = testDF.withColumn('Alone',\
                            when(testDF.FamilySize == 1,1).otherwise(testDF.Alone))


In [17]:
# Creiamo i vettori di feature per il classificatore

# Creiamo la lista degli indici corrispondenti alle colonne categoriche dei due data set
labelDF = trainDF['Pclass','Sex','Embarked','Title','Alone'].unionByName(testDF['Pclass','Sex','Embarked','Title','Alone'])

indexers = [StringIndexer(inputCol = column, outputCol=column+'_index').fit(labelDF) \
                                            for column in ['Pclass','Sex','Embarked','Title','Alone']]

# Usiamo gli indexers per fare il transform delle reali etichette nei due data set separatamente

for indexer in indexers:
    trainDF = indexer.transform(trainDF)
    testDF = indexer.transform(testDF)

# facciamo lo stesso separatamente per l'etichetta di classe

labelIndexer = StringIndexer(inputCol='Survived',outputCol='Survived_index').fit(trainDF)

In [18]:
# Creiamo il mapping inverso per riottenere le etichette originali nelle predizioni a partire dagli indici delle label

labelConverter = IndexToString(inputCol='prediction',outputCol='predictedLabel').setLabels(labelIndexer.labels)

In [19]:
# Creiamo il vettore delle feature

assembler = VectorAssembler().\
                setInputCols(['Age','SibSp','Parch','Fare','FamilySize','Pclass_index','Sex_index','Embarked_index','Title_index','Alone_index']).\
                setOutputCol('Features').setHandleInvalid('keep')

In [20]:
# Creiamo il classificatore
randomForest = RandomForestClassifier().setFeaturesCol('Features').setLabelCol('Survived_index')

In [21]:
# Creiamo la pipeline di addestramento

pipeline = Pipeline().setStages([\
                                    labelIndexer,\
                                    assembler,\
                                    randomForest,\
                                    labelConverter])


In [22]:
# Griglia di ricerca degli iperparametri

paramGrid = ParamGridBuilder().addGrid(randomForest.maxBins,[25, 28, 31])\
                              .addGrid(randomForest.maxDepth,[4,6,8])\
                              .addGrid(randomForest.impurity,['entropy','gini'])\
                              .build()

In [23]:
evaluator = BinaryClassificationEvaluator().setLabelCol('Survived_index').setMetricName('areaUnderPR')

In [24]:
cv = CrossValidator().setEstimator(pipeline)\
                    .setEvaluator(evaluator)\
                    .setEstimatorParamMaps(paramGrid)\
                    .setNumFolds(10)

In [25]:
cvModel = cv.fit(trainDF)

In [1]:
# Eseguiamo la predizione sul test set

predictions = cvModel.transform(testDF)

# Calcoliamo la performance sul training set

performance = cvModel.transform(trainDF)

auprc = evaluator.evaluate(performance)

# Stampiamo la performance
print(f"Area sotto la curva PR: {100*auprc:5.2f}%")

NameError: name 'cvModel' is not defined